# Smoothing Moving Averages

Re-computing moving averages on live data is error-prone, due to close-bias (explain)


In [1]:
# Parameters
symbol = 'BTCUSDT'
start_date = '2021-01-01'
working_interval = '1h'

## Setup

In [2]:
# Imports
import requests
import numpy as np
import pandas as pd

In [3]:
# Fetch Sample Kline Data
start_timestamp = int(pd.to_datetime(start_date).timestamp() * 1000)

base_kline_response = requests.get(f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1m&limit=1000&startTime={start_timestamp}')
base_kline_data = base_kline_response.json()

working_kline_response = requests.get(f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={working_interval}&limit=1000&startTime={start_timestamp}')
working_kline_data = working_kline_response.json()

In [4]:
def make_kline_df(kline_data):
  kline_df = pd.DataFrame(map(lambda kline: kline[:7], kline_data), columns=['open_timestamp','open','high','low','close','volume','close_timestamp'])

  kline_df['open_time'] = pd.to_datetime(kline_df['open_timestamp'], unit='ms')
  kline_df['close_time'] = pd.to_datetime(kline_df['close_timestamp'], unit='ms')

  interval = kline_df.head(1).apply(lambda row: row['close_timestamp'] - row['open_timestamp'] + 1, axis = 1)[0]
  kline_df['interval'] = interval

  kline_df = kline_df.set_index(['open_timestamp', 'close_timestamp'])
  
  return kline_df

In [5]:
base_kline_df = make_kline_df(base_kline_data)
base_kline_df

,,open,high,low,close,volume,open_time,close_time,interval
open_timestamp,close_timestamp,,,,,,,,
1609459200000,1609459259999,28923.63000000,28961.66000000,28913.12000000,28961.66000000,27.45703200,2021-01-01 00:00:00,2021-01-01 00:00:59.999,60000
1609459260000,1609459319999,28961.67000000,29017.50000000,28961.01000000,29009.91000000,58.47750100,2021-01-01 00:01:00,2021-01-01 00:01:59.999,60000
1609459320000,1609459379999,29009.54000000,29016.71000000,28973.58000000,28989.30000000,42.47032900,2021-01-01 00:02:00,2021-01-01 00:02:59.999,60000
1609459380000,1609459439999,28989.68000000,28999.85000000,28972.33000000,28982.69000000,30.36067700,2021-01-01 00:03:00,2021-01-01 00:03:59.999,60000
1609459440000,1609459499999,28982.67000000,28995.93000000,28971.80000000,28975.65000000,24.12433900,2021-01-01 00:04:00,2021-01-01 00:04:59.999,60000
...,...,...,...,...,...,...,...,...,...
1609518900000,1609518959999,29296.30000000,29312.20000000,29282.12000000,29282.12000000,19.99137300,2021-01-01 16:35:00,2021-01-01 16:35:59.999,60000
1609518960000,1609519019999,29282.12000000,29292.20000000,29274.45000000,29288.89000000,18.15801700,2021-01-01 16:36:00,2021-01-01 16:36:59.999,60000
1609519020000,1609519079999,29288.89000000,29303.92000000,29278.72000000,29282.16000000,31.89190600,2021-01-01 16:37:00,2021-01-01 16:37:59.999,60000


In [6]:
working_kline_df = make_kline_df(working_kline_data)

# A bigger interval naturally spans over more time. Filter out the klines that are not encapsulated by the base interval.
base_open_time = base_kline_df.head(1)['open_time'].values[0]
base_close_time = base_kline_df.tail(1)['close_time'].values[0]
working_kline_df = working_kline_df[working_kline_df['open_time'].ge(base_open_time) & working_kline_df['close_time'].le(base_close_time)]

working_kline_df

,,open,high,low,close,volume,open_time,close_time,interval
open_timestamp,close_timestamp,,,,,,,,
1609459200000,1609462799999,28923.63000000,29031.34000000,28690.17000000,28995.13000000,2311.81144500,2021-01-01 00:00:00,2021-01-01 00:59:59.999,3600000
1609462800000,1609466399999,28995.13000000,29470.00000000,28960.35000000,29409.99000000,5403.06847100,2021-01-01 01:00:00,2021-01-01 01:59:59.999,3600000
1609466400000,1609469999999,29410.00000000,29465.26000000,29120.03000000,29194.65000000,2384.23156000,2021-01-01 02:00:00,2021-01-01 02:59:59.999,3600000
1609470000000,1609473599999,29195.25000000,29367.00000000,29150.02000000,29278.40000000,1461.34507700,2021-01-01 03:00:00,2021-01-01 03:59:59.999,3600000
1609473600000,1609477199999,29278.41000000,29395.00000000,29029.40000000,29220.31000000,2038.04680300,2021-01-01 04:00:00,2021-01-01 04:59:59.999,3600000
1609477200000,1609480799999,29220.31000000,29235.28000000,29084.11000000,29187.01000000,1469.95626200,2021-01-01 05:00:00,2021-01-01 05:59:59.999,3600000
1609480800000,1609484399999,29187.01000000,29270.00000000,29077.32000000,29174.35000000,1420.72629100,2021-01-01 06:00:00,2021-01-01 06:59:59.999,3600000
1609484400000,1609487999999,29174.35000000,29191.98000000,28806.54000000,29092.83000000,2380.18091800,2021-01-01 07:00:00,2021-01-01 07:59:59.999,3600000
1609488000000,1609491599999,29092.84000000,29178.03000000,28872.24000000,29000.01000000,2008.16573900,2021-01-01 08:00:00,2021-01-01 08:59:59.999,3600000
